## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("../WeatherPy_database.csv")


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kirakira,SB,-10.4544,161.9205,77.16,89,97,9.33,overcast clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,85.71,69,85,2.42,overcast clouds
14,14,Sinnamary,GF,5.3833,-52.9500,81.95,78,55,9.37,broken clouds
19,19,Cabo San Lucas,MX,22.8909,-109.9124,87.80,69,1,3.00,clear sky
20,20,Albany,US,42.6001,-73.9662,80.46,72,81,3.00,broken clouds
24,24,Waingapu,ID,-9.6567,120.2641,77.11,82,47,13.02,scattered clouds
25,25,Tual,ID,-5.6667,132.7500,80.04,84,83,19.24,broken clouds
28,28,Umm Lajj,SA,25.0213,37.2685,90.97,61,0,13.20,clear sky
30,30,Inirida,CO,3.8653,-67.9239,81.55,74,100,1.05,overcast clouds
32,32,Champerico,GT,14.3000,-91.9167,89.35,81,87,1.57,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                292
City                   292
Country                290
Lat                    292
Lng                    292
Max Temp               292
Humidity               292
Cloudiness             292
Wind Speed             292
Current Description    292
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                290
City                   290
Country                290
Lat                    290
Lng                    290
Max Temp               290
Humidity               290
Cloudiness             290
Wind Speed             290
Current Description    290
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
     # 6c. Get latitude and longitude from DataFrame 
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        pass


In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kirakira,SB,77.16,overcast clouds,-10.4544,161.9205,SanBiz Lodge
3,Ponta Do Sol,PT,85.71,overcast clouds,32.6667,-17.1000,Hotel do Campo
14,Sinnamary,GF,81.95,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
19,Cabo San Lucas,MX,87.80,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
20,Albany,US,80.46,broken clouds,42.6001,-73.9662,


In [12]:
hotel_df.count()

City                   290
Country                290
Max Temp               290
Current Description    290
Lat                    290
Lng                    290
Hotel Name             290
dtype: int64

In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df['Hotel Name'] == ''] = np.nan
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.count()

City                   258
Country                258
Max Temp               258
Current Description    258
Lat                    258
Lng                    258
Hotel Name             258
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [17]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))